# General Image Maker for for Browning Trail Cameras

---

This worksheet contains tools in this repository to build *.BRN firmware files for Browning trail camera. Specific target cameras and features are summarized below.  The "build" process contains elements of Python, Make, and some hand steps required to synchronize with Ghidra images




## Option Summary


| Camera Model | Factory Baseline  | Current WBWL | Version  | Build Date |
|--------------|-------------------|--------------|----------|------------|
| BTC-7A       | NA           |  pointer     | version  |  date      |
| BTC-8A       | NA           |  pointer     | version  |  date      |
| BTC-7E       | NA           |  pointer     | version  |  date      |
| BTC-8E       | NA           |  pointer     | version  |  date      |
| BTC-7E-HP4   | NA           |  pointer     | version  |  date      |
| BTC-8E-HP4   | NA           |  pointer     | version  |  date      |
| BTC-7E-HP5   | NA           |  pointer     | version  |  date      |
| BTC-8E-HP5   | NA           |  pointer     | version  |  date      |


Current Feature List:
 * Custom Ribbon: In "Playback" menu, the date and time of the photo/video are shown is larger font at the bottom of the playback screen, making it easier to tell when the image was taken when reviewing in the field.
 * Night Video Limit: The default manufacturer firmware limits night time (flash illuminated) videos to 20 Seconds.  This option removes that limit, allowing illuminated videos of the same duration as daylight videos.
 * Custom Info Strip: Adds "seconds" to the time of day; adds battery percent; compresses the size of the browning logo so it doesn't extend up into the main screen space.  
 * Custom Volume/Directory/Filenames: Normally, this camera renames the SD card "volume"  to "BROWNING", in folders in DCIM called xxxBTCF, in files called IMG_xxxx.  With this modification, these labels are all taken from the camera name, as set by the user.  Note that only the first 5 and 4 characters are used for the folder and file suffix/prefix respectively.  Thus, for camera name, "MYCAMERA", the SD card will be labeled "MYCAMERA" and files can be found in DCIM/100MYCAME/MYCA0001.JPG.  Any "space" in the first 5 characters is replaced by an "underscore".
 * DSLR Trigger: Causes the "aim led" to turn on whenever a photo or video is taken.  This LED can be used to trigger a DSLR camera, while also allowing the trail camera to take photos or video.
 * Extended SD Power: Causes the CPU and SD card to remain powered on for 30 seconds after each photo or video.  This to allow shared SD card to be read by another device.  Note that during this time, the camera will not trigger.
 * Time and Date Format Options: Added options for time format -- 12H/24H; and date formats MM/DD/YYYY; DD/MM/YYYY; YYYYMMDD.  The time format also applies to the set date/time menu, allowing time to set in 24-hour format (vs. AM/PM)
 * IR Power: Added an "Off" setting which prevents the camera from using any IR illumination for black and white night images.  For use with external (IR) light sources.
 * Daytime Thereshold: Shifts the threshold at which camera takes day vs. night pictures down to allow color photos with less light.  There is an "No Light" option which causes the camera to take color images, without illumination, even when dark.  Can be used with external (white) light sources.
 * Timelapse File: The factory firmware stores photos captured in Timelapse+ mode in large video files (actually avi format) in a directory called "TLS" and in files with .tls suffix).  I've added a new menu: "Timelapse File". In this new menu, selecting ".TLS" will create .tls files, while selecting ".JPG" causes the camear to store files captured in Timelapse+ mode as single JPG images.  This allows timelapse photos to be viewed in the camera, and enables per-file post processing.  



### Mounting My Google Drive
The method is described under this Colab code snippet: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA. Never give out your account username and password. Read this Colab code snippet to understand how this connection is made and authenticated. There are other ways to connect your Google Drive or upload your data if you do not find this method suitable.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%env REPOSITORY_BASE=/content/drive/MyDrive/local_repositories/unified-btc-reverse/

env: REPOSITORY_BASE=/content/drive/MyDrive/local_repositories/unified-btc-reverse/


In [3]:
import sys
import os
sys.path.insert(1,os.getenv('REPOSITORY_BASE')+'/tools/python/')

## Importing Python Tools
Import tools for: defining structures; parsing EEPROM files; applying patches to binary code; and creating a new .BRN file

Also includes dict database of all targets and their parameters

In [4]:
!pip install python-snappy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.2 MB/s eta 0:00:00


In [5]:
import math
import Struct
import carve_BTC_BRN
import combineFirmware
import codePatcher
import BTC_EEPROM_Parser
import BTC_EEPROM_Writer
import BTC_BURN_Maker
import BTC_Doc_Maker
import UTC_Symbol_Maker
import SST_Parser
import Build_Database
import Mx25EepromParser

In [6]:
import imp
imp.reload(codePatcher)
imp.reload(carve_BTC_BRN)
imp.reload(BTC_BURN_Maker)
imp.reload(BTC_Doc_Maker)
imp.reload(UTC_Symbol_Maker)
imp.reload(BTC_EEPROM_Parser)
imp.reload(BTC_EEPROM_Writer)
imp.reload(SST_Parser)
imp.reload(Build_Database)
imp.reload(Mx25EepromParser)


<module 'Mx25EepromParser' from '/content/drive/MyDrive/local_repositories/unified-btc-reverse//tools/python/Mx25EepromParser.py'>

# Carving .BRN File
This operation extracts the various "offsets" from the .BRN file.  We need Offset2.A for file system additions (e.g. SST files); Offset3 for Ghidra work

In [ ]:
from carve_BTC_BRN import carveBTCBRN
from Build_Database import g_wbwl_btc_targets
##camera_targets = ['BTC-7E', 'BTC-7E-HP4', 'BTC-8E-HP4']
#camera_targets = ['BTC-7E-HP4']
camera_targets = ['BTC-PATRIOT-FHD']

for target in camera_targets:
  carver = carveBTCBRN()
  if 'Factory BRN Filename' in g_wbwl_btc_targets[target]:
    print(f'Carving files for Target: {target}')
    target_dir = g_wbwl_btc_targets[target]['Target Directory']
    factory_firmwware_ldir = g_wbwl_btc_targets[target]['Factory BRN Dir']
    factory_firmwware_lfilename = g_wbwl_btc_targets[target]['Factory BRN Filename']
    factory_firmware_dir =  os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/factory-firmware-images/' + factory_firmwware_ldir + '/'
    factory_firmware_filename = factory_firmware_dir + factory_firmwware_lfilename

    carver.carve_firmware(factory_firmware_filename, factory_firmware_dir)



Carving files for Target: BTC-PATRIOT-FHD
Info::carve: (A) partition offset = 0x96920
Info::carve A size: 0x   40000
Info::carve: (B) partition offset = 0xd6920
Info::carve B size: 0x   40000


# Recreating an EEPROM Image
When hacking so closely to the bits, it's wise to have a safety net.  I've found it's really easy to "brick" these cameras, or to leave them without a way of loading new (working) firmware from an SD card.  For this reason, I use a dedicated development camera in which I have removed the EEPROM and replaced with a removable ZIFF socket.  As long as I have a "good" impage for an  EEPROM, I can drop it in the ZIFF socket, and recover from firmware mistakes.  

In this section, we generate an EERPOM binary file from the .BRN image, and from the EEPROM header (which is not included in the .BRN image).  I get the EEPROM header by "sniffing" it off the EEPROM in a working camera during power on with a logic analyzer *before* I remove the EEPROM.  The heat required to unsolder the EEPROM almost always leaves is empty, and often unable to store data.

A few notes:
* Unless you use exactly the same EEPROM part (which may not be possible), you can't just make a copy of the factory EEPROM image for another EEPROM.  This is because the EEPROM header contains the device ID of the EEPROM type.  This gets checked during boot, and if the ID read from the EEPROM doesn't match what's been stored in teh header block, it will not load.
* You can't just change values in the EEPROM header.  THe header is protected by a checksum, which must also be updated

### Sniffing EEPROM Header with Logic Analyzer
There are a few kbytes of header information in the EEPROM which are not encoded in the .BRN image.  I suspect they are target SOC-specific, and may contain data used to setup low level hardware, like the DRAM controller. In a pinch, these can be cribbed from images of other platforms, but it is easiest to get this header info from the target under development by "sniffing" the EEPROM signals during system power on (and boot). A low-cost USB logic analzyer (e.g. Kingst LA5016) turns the trace into a .CSV file.  This file, in turn, is parsed by utiliites below to produce a binary file.

Note that this binary file only contains data that was read from the EEPROM.  In Browning Trail cameras, at least, this is not the whole EEPROM image.  Instated, it is just the executable code segment.  The A and B FAT12 File systems hosted on the EEPROM are "demand" loaded.

I have also had some trouble getting accurate data from my low cost logic analyzer at the high data speeds used to transfer the bulk of the EEPROM image (to save time and reduce Trigger speed).  Glitches in the logic analyser trace, especially of the clock signal, confuse the parser, resulting in missing segments in the binary.

My goal is to figure out where in the EEPROM the A and B file systems are stored. (I can't seem to figure out how, or whetehr, this information is encoded in the header file)

In [ ]:
from Mx25EepromParser import Mx25EepromParser
from Build_Database import g_wbwl_btc_targets

##camera_targets = ['BTC-7E', 'BTC-7E-HP4', 'BTC-8E-HP4']
#camera_targets = ['BTC-7E-HP4']
camera_targets = ['BTC-PATRIOT-FHD']

#for target in g_wbwl_btc_targets:
for target in camera_targets:
  if 'LA File' in g_wbwl_btc_targets[target]:
    lc_target = target.lower()
    la_filebase = g_wbwl_btc_targets[target]['LA File']
    ep = Mx25EepromParser()
    ep.image_file_from_la_trace(os.getenv('REPOSITORY_BASE')+'targets/' + lc_target + '/logic-analyzer-traces/', la_filebase + '.csv', la_filebase + '.text', la_filebase + '.bin')

In [ ]:
#camera_targets = ['BTC-7E', 'BTC-7E-HP4', 'BTC-8E-HP4']
##camera_targets = ['BTC-8E', 'BTC-8E-HP5']
##camera_targets = ['BTC-8E-HP5']
camera_targets = ['BTC-PATRIOT-FHD']
#

#for target in g_wbwl_btc_targets:
for target in camera_targets:
  if 'EEPROM Dir' in g_wbwl_btc_targets[target]:
    lc_target = target.lower()
    eeprom_dir = g_wbwl_btc_targets[target]['EEPROM Dir']
    eeprom_file = g_wbwl_btc_targets[target]['EEPROM Filename']
    file_path = os.getenv('REPOSITORY_BASE')+'targets/' + lc_target + '/eeprom-images/' + eeprom_dir + '/' + eeprom_file
    with open (file_path, 'rb') as eef:
      print(target)
      step = 2
      for i in range(0,256, step):
        data = eef.read(step)
        int_data = int.from_bytes(data, byteorder='little')
        if step == 1:
          print(f'{i:04x} {int_data:02x}')
        elif step == 2:
          print(f'{i:04x} {int_data:04x}')
        else:
          print(f'{i:04x} {int_data:08x}')

FileNotFoundError: ignored

### Writing a New EEPROM Image

In [ ]:
from BTC_EEPROM_Writer import BTC_EEPROM_Writer
writer=BTC_EEPROM_Writer()
writer.calculate_file_checksum("/content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-8e-hp5/eeprom-images/2022-09-04-Factory-Baseline/2020-09-04-Post-L10200F.bin", buffer_size=256)

Checksum for /content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-8e-hp5/eeprom-images/2022-09-04-Factory-Baseline/2020-09-04-Post-L10200F.bin is 2E7FFD3A; Num elements = 64; Size/4 = 64.0


780139834

In [ ]:
from BTC_EEPROM_Writer import BTC_EEPROM_Writer
from Build_Database import g_wbwl_btc_targets

#camera_targets = ['BTC-7E', 'BTC-7E-HP4', 'BTC-8E-HP4']
#camera_targets = ['BTC-7E']
camera_targets = ['BTC-PATRIOT-FHD']


#for target in g_wbwl_btc_targets:
for target in camera_targets:
  if 'EEPROM Dir' in g_wbwl_btc_targets[target]:
    writer = BTC_EEPROM_Writer()
    lc_target = target.lower()
    la_file = g_wbwl_btc_targets[target]['LA File']
    brn_dir = g_wbwl_btc_targets[target]['Factory BRN Dir']
    eeprom_dir = g_wbwl_btc_targets[target]['EEPROM Dir']
    eeprom_file = g_wbwl_btc_targets[target]['EEPROM Filename']
    writer.write_eeprom_binary(os.getenv('REPOSITORY_BASE')+'targets/' + lc_target + '/logic-analyzer-traces/' + la_file + '.bin',
                               os.getenv('REPOSITORY_BASE')+'targets/' + lc_target + '/factory-firmware-images/' + brn_dir + '/',
                               os.getenv('REPOSITORY_BASE')+'targets/' + lc_target + '/eeprom-images/' + eeprom_dir + '/' + eeprom_file,
                               target)

Checksum for /content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-patriot-fhd/logic-analyzer-traces/2024-01-19-Patriot-EEPROM-Boot.bin is 99143D6E; Num elements = 60; Size/4 = 60.0
info::write_eeprom_header -- checksum is 0x99143d6e; inverted checksum: 0x66ebc291
eeprom_header_length = 12288
eeprom output filelen = 12288
info::write_segment: writing 874 bytes to offset 0x00000100
info::write_eeprom_binary: writing app from /content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-patriot-fhd/factory-firmware-images/2021-11-04-Baseline/offset3 to address 0x00003000
info::write_segment: writing 2518528 bytes to offset 0x00003000
info::write_eeprom_binary: writing a file system from /content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-patriot-fhd/factory-firmware-images/2021-11-04-Baseline/offset2.A to address 0x00380000
info::write_segment: writing 262144 bytes to offset 0x00380000
info::write_eeprom_binary: writing b file system 

In [ ]:
! ls /content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-7e/eeprom-images/2023-08-13-Synthesized-Baseline/


2023-08-13-BTC-7E-EEPROM-Image.bin


# Augmenting .SST Files
Starting with the Edge, Browning supports internationalization of the menu system.  The firmware contains numerical references to strings which comprise the menu items.  These strings are loaded, along with per-language fonts, when a new langauge is chosen from the "Language" menu.  

THe files are labeled "x.SST" but, on inspection, have a header with starts out "RIFF".  Per Wikipedia, "RIFF" is a format for holding variable length chunks of arbitrary data.  Nominally -- 4-byte ascii identifer, 4-byte length field, and chunk of that length.  

The file also contains an ASCII string "SNAP" just after the 48-byte header.  When this string is present, the firmware apparently invokes a routine called, "snappy" to process the file.  

I use a python version of "snappy" (a high throughoput compression/decompression library) to take the .SST files apart, add the new strings required for new menu itmes, and then put the SST files back together

## Extracting SST files from A File System
This is a manual step in which I use a file-system tool -- like PowerISO -- to load the image for the A file system contained in offset2.A.  I then use PowerISO to extract the SST files for each of the camera targets into the directory: \unified-btc-reverse\targets\{target}\factory-firmware-images\{factory-firmware-directory}

The files can be found in A:/RO_RES/UI/SST/ and include: DEUTCH.sst, DUTCH.sst, ENGLISH.SST, ESPANOL.sst, FRANCIS.sst, ITALINO.sst, and POLISH.sst


In [ ]:
from SST_Parser import SST_Parser
from Build_Database import g_wbwl_btc_targets

parser = SST_Parser()
for target in g_wbwl_btc_targets:
  if 'Strings' in g_wbwl_btc_targets[target]:
    print(f'Generating Language files for Target: {target}')
    target_dir = g_wbwl_btc_targets[target]['Target Directory']
    factory_firmwware_ldir = g_wbwl_btc_targets[target]['Factory BRN Dir']
    factory_firmware_dir =  os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/factory-firmware-images/' + factory_firmwware_ldir + '/'
    language_dict = g_wbwl_btc_targets[target]['Strings']
    parser.generate_language_files(factory_firmware_dir,
                                   os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/file-system-additions/LanguageSSTFiles/',
                                   language_dict)


Generating Language files for Target: BTC-7E
Generating Language files for Target: BTC-8E
Generating Language files for Target: BTC-7E-HP4
Generating Language files for Target: BTC-8E-HP4
Generating Language files for Target: BTC-7E-HP5
Generating Language files for Target: BTC-8E-HP5


In [ ]:
from SST_Parser import SST_Parser
from Build_Database import g_wbwl_btc_targets

parser = SST_Parser()

print(f'Generating Language Include Files')

parser.generate_include_file(g_wbwl_btc_targets,
                            os.getenv('REPOSITORY_BASE'),
                            os.getenv('REPOSITORY_BASE') + 'include/SST-strings.h')

Generating Language Include Files


In [ ]:
!head -1000 /content/drive/MyDrive/local_repositories/unified-btc-reverse/include/SST-strings.h

## Putting the Updated .SST files into a New Image for A File system
Atter we have created new SST files, left in
\unified-btc-reverse\targets\{target}\file-system-additions\LanguageSSTFiles\
we can now use a tool like PowerISO to update the A file systemn image to include these new files.  In order to keep PowerISO from crashing (because of space over-run), I've found it necessary to delete the files to be replaced instead of just over-writing them.

Edit the image in ../targets/btc-xx-xxx/A-FileSystem/offset2.A.wbwl.img to include all of the geneated .sst files.

## Other File System Hacks
Note that the A file system contains all of the JPG and icon (SFN) files used in the GUI. I have not yet attacked the SFN file format, but the JPG files are standard JPG, optimized for the 480 x 640 color LCD screen.  The file named, PMT_BK.JPG is displayed on the "splash" screen during during camera boot.  In principle, any/all of these files could be overwritten.  Keep in mind, though, the space restrictions on this small file system, and aim to match the size of new files to the ones they replace.  I found that PowerISO behaves better (doesn't crash) if I first delete the file I want to replace, and then copy the new file into its place.

### Creating 2A file system (Manual Step)
I don't have an automated way of creating a FAT12 file system that works with these images.  Instead, I use PowerISO on a PC.  I manually delete old, and insert new SST files, as well as the JPG Splash Photo.  If you forget this step, new menu items will show up without any text (since there are no strings to fill in)

# Loading Patch Lists
These are hand created python objects which specify entry points and in-place patches.  There is one patch list, with provisions for different values for different camera targets where necessary.

In [7]:
sys.path.insert(1, os.getenv('REPOSITORY_BASE')+"hand-patches/")

In [16]:
import btc_patch_list

imp.reload(btc_patch_list)


from btc_patch_list import firmware_bundle_a_patch_list
from btc_patch_list import night_video_limit_patch_list
from btc_patch_list import custom_ribbon_patch_list
from btc_patch_list import custom_info_strip_patch_list
from btc_patch_list import dslr_trigger_patch_list
from btc_patch_list import utilities_patch_list
from btc_patch_list import vfn_patch_list
from btc_patch_list import extended_SD_power_patch_list
from btc_patch_list import menus_patch_list
from btc_patch_list import timelapse_patch_list
from btc_patch_list import aperture_patch_list
from btc_patch_list import pressure_temperature_patch_list
from btc_patch_list import ir_flash_power_patch_list
from btc_patch_list import cdt_menu_patch_list
from btc_patch_list import pt_patch_list
from btc_patch_list import rsc_patch_list
from btc_patch_list import fdb_patch_list
from btc_patch_list import set_version_string




### Checking Hand Generated Patch List
The patch list file has grown quite large, and it's built by hand.  Run some consistency checks, at least, to make sure it's not completely broken

In [ ]:

from codePatcher import codePatcher
patcher = codePatcher()

patch_lists = [firmware_bundle_a_patch_list, night_video_limit_patch_list, custom_ribbon_patch_list, custom_info_strip_patch_list,
               dslr_trigger_patch_list, utilities_patch_list, vfn_patch_list ,extended_SD_power_patch_list, menus_patch_list,
               timelapse_patch_list, aperture_patch_list, pressure_temperature_patch_list]
for patch_list in patch_lists:
  patcher.consistency_check(patch_list)

# Compiling Source Code and Creating Binaries

---

Compile all the source code modules and package them up into a series of binaries specific to each camera target.   Note that the source code covers all camera models.  Model specific data structrures, enums, code options, are all covered by target-specific #ifdefs in the source code.   


## Installing GCC Suite
Start by updating local packages; then install gcc-9-mips

In [9]:
%%capture
!sudo apt update
!sudo apt install gcc-9-mips-linux-gnu
!mips-linux-gnu-gcc-9 --version

##Compiling and Linking Source Code
Make is used to compile and link the code.  This is done in two steps -- all_part_1, which results in symbols and object files; and all_part_2 which results in a series of bytes files suitable for use by the binary code patcher.   In between these steps is the manual step of creating a .cmd file for the linker which defines all of the functions and variables used by the new code.  A Ghidra script, running on the annotated binary of each target camera, creates this .cmd file.

In [10]:
os.chdir(os.getenv('REPOSITORY_BASE')+ 'src')

In [11]:
! make all_clean

cd ../targets/btc-7e/bin && make clean
make[1]: Entering directory '/content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-7e/bin'
rm -f *.s *.o *.symbols part_1 part_2
make[1]: Leaving directory '/content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-7e/bin'
cd ../targets/btc-8e/bin && make clean
make[1]: Entering directory '/content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-8e/bin'
rm -f *.s *.o *.symbols part_1 part_2
make[1]: Leaving directory '/content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-8e/bin'
cd ../targets/btc-7e-hp4/bin && make clean
make[1]: Entering directory '/content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-7e-hp4/bin'
rm -f *.s *.o *.symbols part_1 part_2
make[1]: Leaving directory '/content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-7e-hp4/bin'
cd ../targets/btc-8e-hp4/bin && make clean
make[1]: Entering directory '/content/drive/MyDrive

In [13]:
!make all_part_1

cd ../targets/btc-7e/bin && make part_1
make[1]: Entering directory '/content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-7e/bin'
mips-linux-gnu-gcc-9 -I../../../include -Os -nostdlib -mno-abicalls -march=r4000 -EL -S -DBTC_7E -o entry0.s ../../../src/entry0.c
mips-linux-gnu-gcc-9 -march=r4000 -mno-abicalls  -c -EL entry0.s -o entry0.o
mips-linux-gnu-objdump -t entry0.o > entry0.symbols
mips-linux-gnu-gcc-9 -I../../../include -Os -nostdlib -mno-abicalls -march=r4000 -EL -S -DBTC_7E -o entry1.s ../../../src/entry1.c
mips-linux-gnu-gcc-9 -march=r4000 -mno-abicalls  -c -EL entry1.s -o entry1.o
mips-linux-gnu-objdump -t entry1.o > entry1.symbols
mips-linux-gnu-gcc-9 -I../../../include -Os -nostdlib -mno-abicalls -march=r4000 -EL -S -DBTC_7E -o entry2.s ../../../src/entry2.c
mips-linux-gnu-gcc-9 -march=r4000 -mno-abicalls  -c -EL entry2.s -o entry2.o
mips-linux-gnu-objdump -t entry2.o > entry2.symbols
mips-linux-gnu-gcc-9 -I../../../include -Os -nostdlib -mno-abicalls -

In [ ]:
! cd ../targets/btc-7e-hp5/bin && grep -F 'HceTaskBoot' timelapse.symbols

000003ac g     F .text	0000015c tls_HceTaskBoot2Cap_Task0
00000000         *UND*	00000000 HceTaskBoot2Cap_Task0


In [ ]:
! cd ../targets/btc-7e-hp5/bin && grep -F 'snapYuv2' general.cmd

   snapYuv2ExifJpgWrite = 0x80182488;


### Resolving Symbols
The C code can reference any of the global variables or functions defined in the factory binary.  Ghidra knows where these are.  I have written a script in Java that takes the .symbol file generated above, and creates a .cmd file to pass to the linker, below.  .cmd file includes symbol definitions as well as gp value (also extracted from Ghidra symbol table).  It also creates a .bytes file with the contents of functions which are being replaced (this to allow a check during binary build time that patches are being applied in the correct place in the binary).

### Part 2 of Make
Make a series of binaries with the correct collection of features.  These need the platform-specific targets dictionaries from Ghidra, we we have to run this second half manually

In [14]:
!make all_part_2

cd ../targets/btc-7e/bin && make part_2
make[1]: Entering directory '/content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-7e/bin'
mips-linux-gnu-ld --warn-multiple-gp -G65535 --warn-section-align -EL -T general.cmd -o release.bin entry0.o entry1.o entry2.o entry3.o entry4.o custom-ribbon.o custom-info-strip.o volume-file-naming.o extend-event-SD-card.o rtc-formats.o dslr-trigger.o menus.o timelapse.o aperture.o aperture-menu.o ir-flash-menu.o ir-flash-menu-early.o custom-set-date-time.o reduce-sd-clock.o 
mips-linux-gnu-ld: warning: start of section .text changed by 12
mips-linux-gnu-ld: warning: start of section .data changed by 12
mips-linux-gnu-ld --warn-multiple-gp -G65535 --warn-section-align -EL -T general.cmd -o test.bin entry0.o entry1.o entry2.o entry3.o entry4.o custom-ribbon.o custom-info-strip.o volume-file-naming.o extend-event-SD-card.o rtc-formats.o dslr-trigger.o menus.o timelapse.o aperture.o aperture-menu.o ir-flash-menu.o ir-flash-menu-early.o cu

In [ ]:
! cd ../targets/btc-8e-hp4/bin && make part_2

make: 'part_2' is up to date.


In [ ]:
! cd ../targets/btc-8e-hp5/bin && cat general.cmd | grep rtc_extra


   set_rtc_extra_operation_mode = 0x8005bb8c;
   set_rtc_extra_byte_range = 0x80060b38;
   get_rtc_extra_operation_mode = 0x8005bbdc;
   get_rtc_extra_byte_range = 0x80060b80;


In [ ]:
! cd ../targets/btc-7e-hp5/bin && mips-linux-gnu-objcopy -H -S -O binary  test.bin test.bytes

## Build Images
Use the binaries generated above to create .BRN images in the right places with the various set of features.

### Generate Version Table
Creates a table of factory and newly released binaries in .md format that can be pasted into "README.md" file

In [17]:
from Build_Database import g_wbwl_btc_targets
imp.reload(Build_Database)


from BTC_BURN_Maker import BTC_BURN_Maker
from BTC_Doc_Maker import BTC_Doc_Maker

camera_targets = ['BTC-7A', 'BTC-8A', 'BTC-7E', 'BTC-8E', 'BTC-7E-HP4', 'BTC-8E-HP4', 'BTC-7E-HP5', 'BTC-8E-HP5', 'BTC-PATRIOT-FHD']
#camera_targets = ['BTC-7E-HP5', 'BTC-8E-HP5']
#camera_targets = ['BTC-7E-HP5']

# Make the table of available firmware
doc_maker = BTC_Doc_Maker()
doc_dir = os.getenv('REPOSITORY_BASE')
doc_filename = "version_table.md"
eeprom_filename = "eeprom_table.md"
doc_maker.file_print_version_table(doc_dir, doc_filename, g_wbwl_btc_targets, camera_targets)
doc_maker.file_print_eeprom_table(doc_dir, eeprom_filename, g_wbwl_btc_targets, camera_targets)
# Set the version
set_version_string(g_wbwl_btc_targets)

Extended Filename = /content/drive/MyDrive/local_repositories/unified-btc-reverse/version_table.md
Extended Filename = /content/drive/MyDrive/local_repositories/unified-btc-reverse/eeprom_table.md
camera_target: BTC-PATRIOT-FHD
camera_target: BTC-7A
camera_target: BTC-8A
camera_target: BTC-7E
camera_target: BTC-8E
camera_target: BTC-7E-HP4
camera_target: BTC-8E-HP4
camera_target: BTC-7E-HP5
camera_target: BTC-8E-HP5


### TEST
Used during debug so I don't clobber any of the released binaries

In [18]:
from Build_Database import g_wbwl_btc_targets
from Build_Database import g_btc_major_version
from BTC_BURN_Maker import BTC_BURN_Maker
from BTC_Doc_Maker import BTC_Doc_Maker

print (f'Building version {g_btc_major_version}')
camera_targets = ['BTC-7E', 'BTC-8E', 'BTC-7E-HP4', 'BTC-8E-HP4', 'BTC-7E-HP5', 'BTC-8E-HP5']
#camera_targets = ['BTC-7E-HP5', 'BTC-8E-HP5']
#camera_targets = ['BTC-7E-HP5']
#camera_targets = ['BTC-7E']
#camera_targets = ['BTC-8E']
#camera_targets = ['BTC-8E-HP4']
#camera_targets = ['BTC-8E-HP5']

# Set the version
set_version_string(g_wbwl_btc_targets)



#for target in g_wbwl_btc_targets:
for target in camera_targets:
  burn_maker = BTC_BURN_Maker()
  if 'Factory BRN Filename' in g_wbwl_btc_targets[target]:
    print(f'Building .BRN file for Target: {target}')
    target_dir = g_wbwl_btc_targets[target]['Target Directory']
    factory_firmware_ldir = g_wbwl_btc_targets[target]['Factory BRN Dir']
    factory_firmware_filename = g_wbwl_btc_targets[target]['Factory BRN Filename']
    eeprom_ldir = g_wbwl_btc_targets[target]['EEPROM Dir']
    factory_firmware_dir =  os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/factory-firmware-images/' + factory_firmware_ldir + '/'
    dest_firmware_filename = factory_firmware_filename
    dest_dir = os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/created-burn-images/' + 'TEST/'
    eeprom_dir = os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/eeprom-images/' + eeprom_ldir
    eeprom_filename = g_wbwl_btc_targets[target]['EEPROM Filename']
    cmd_directory = os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/bin/'
    fs_directory_name = os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/A-File-System/offset2.A.wbwl.img'

    burn_maker.make_burn_file(target=target,
                            ref_brn_directory=factory_firmware_dir,
                            ref_brn_filename= factory_firmware_filename,
                            dest_directory=dest_dir,
                            dest_file=dest_firmware_filename,
                            ref_eeprom_directory=eeprom_dir,
                            ref_eeprom_filename=eeprom_filename,
                            fs_directory_name=fs_directory_name,
                            patch_directory=os.getenv('REPOSITORY_BASE')+"hand-patches/",
                            cmd_directory=cmd_directory,
                            list_of_patch_lists=[
                                                custom_ribbon_patch_list,
                                                night_video_limit_patch_list,
                                                custom_info_strip_patch_list,
                                                firmware_bundle_a_patch_list,
                                                timelapse_patch_list,
                                                vfn_patch_list,
                                                dslr_trigger_patch_list,
                                                menus_patch_list,
                                                extended_SD_power_patch_list,
                                                ir_flash_power_patch_list,
                                                aperture_patch_list,
                                                cdt_menu_patch_list,
                                                pt_patch_list,
                                                rsc_patch_list
                                                #fdb_patch_list
                                                ],
                            list_of_patch_bases = ["test"], command_file_base = "general",
                            offset2a_source='HAND_PATCH'
                            )


Building version 240506P
Building .BRN file for Target: BTC-7E
Info::carve: (A) partition offset = 0x96920
Info::carve A size: 0x  160000
Info::carve: (B) partition offset = 0x1f6920
Info::carve B size: 0x   20000
Info::make_burn_file: getting A file system from /content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-7e/A-File-System/offset2.A.wbwl.img
Info::make_burn_file: adding patch_list: dict_keys(['Custom_Ribbon', 'ld_draw_video_scroll_bar_hook', 'ld_draw_video_scroll_bar_hook2', 'ld_clear_video_scroll_bar_hook'])
Info::make_burn_file: adding patch_list: dict_keys(['No_Video_Limit_Store'])
Info::make_burn_file: adding patch_list: dict_keys(['date_local_sprintf', 'time_local_sprintf', 'last_strlen', 'still_logo', 'video_logo', 'load_font_width'])
Info::make_burn_file: adding patch_list: dict_keys(['firmware_ID0', 'firmware_ID1'])
Info::make_burn_file: adding patch_list: dict_keys(['decoder_function', 'tlps_function_count', 'tlps_index_argument', 'tlps_fsm', 'tlps

In [ ]:
!pwd

/content/drive/MyDrive/local_repositories/unified-btc-reverse/src


In [ ]:
!ls -alst ../targets/btc-8e-hp5/created-burn-images/TEST/

total 9069
4534 -rw------- 1 root root 4642147 Mar 16 15:29 brnbtc82.BRN
 598 -rw------- 1 root root  611840 Mar 16 15:29 offset0
   4 -rw------- 1 root root    4096 Mar 16 15:29 offset1
 712 -rw------- 1 root root  729088 Mar 16 15:29 offset2.A
  40 -rw------- 1 root root   40960 Mar 16 15:29 offset2.B
   1 -rw------- 1 root root     288 Mar 16 15:29 offset2.header
3178 -rw------- 1 root root 3253760 Mar 16 15:29 offset3
   1 -rw------- 1 root root     704 Mar 16 15:29 offset5
   1 -rw------- 1 root root     883 Mar 16 15:29 offset6
   1 -rw------- 1 root root     884 Mar 16 15:29 offset6.padded
   1 -rw------- 1 root root      16 Mar 16 15:29 prometheus_trailer
   1 -rw------- 1 root root     512 Mar 16 15:29 SPHOST.header


### RELEASE
Create release version of images

In [19]:
from Build_Database import g_wbwl_btc_targets
from BTC_BURN_Maker import BTC_BURN_Maker

camera_targets = ['BTC-7E', 'BTC-8E', 'BTC-7E-HP4', 'BTC-8E-HP4', 'BTC-7E-HP5', 'BTC-8E-HP5']
##camera_targets = ['BTC-8E-HP5']

print (f'Buidling version {g_btc_major_version}')
# Make the table of available firmware
doc_maker = BTC_Doc_Maker()
doc_dir = os.getenv('REPOSITORY_BASE')
doc_filename = "version_table.md"
doc_maker.file_print_version_table(doc_dir, doc_filename, g_wbwl_btc_targets, camera_targets)
# Set the version
set_version_string(g_wbwl_btc_targets)

#for target in g_wbwl_btc_targets:
for target in camera_targets:
  burn_maker = BTC_BURN_Maker()
  if 'Factory BRN Filename' in g_wbwl_btc_targets[target]:
    print(f'Building .BRN file for Target: {target}')
    target_dir = g_wbwl_btc_targets[target]['Target Directory']
    factory_firmware_ldir = g_wbwl_btc_targets[target]['Factory BRN Dir']
    factory_firmware_filename = g_wbwl_btc_targets[target]['Factory BRN Filename']
    eeprom_ldir = g_wbwl_btc_targets[target]['EEPROM Dir']
    factory_firmware_dir =  os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/factory-firmware-images/' + factory_firmware_ldir + '/'
    dest_firmware_filename = factory_firmware_filename
    dest_dir = os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/created-burn-images/' + 'RELEASE/'
    eeprom_dir = os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/eeprom-images/' + eeprom_ldir
    eeprom_filename = g_wbwl_btc_targets[target]['EEPROM Filename']
    cmd_directory = os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/bin/'
    fs_directory_name = os.getenv('REPOSITORY_BASE')+'targets/' + target_dir + '/A-File-System/offset2.A.wbwl.img'

    burn_maker.make_burn_file(target=target,
                            ref_brn_directory=factory_firmware_dir,
                            ref_brn_filename= factory_firmware_filename,
                            dest_directory=dest_dir,
                            dest_file=dest_firmware_filename,
                            ref_eeprom_directory=eeprom_dir,
                            ref_eeprom_filename=eeprom_filename,
                            fs_directory_name=fs_directory_name,
                            patch_directory=os.getenv('REPOSITORY_BASE')+"hand-patches/",
                            cmd_directory=cmd_directory,
                            list_of_patch_lists=[
                                                custom_ribbon_patch_list,
                                                night_video_limit_patch_list,
                                                custom_info_strip_patch_list,
                                                firmware_bundle_a_patch_list,
                                                timelapse_patch_list,
                                                vfn_patch_list,
                                                dslr_trigger_patch_list,
                                                menus_patch_list,
                                                extended_SD_power_patch_list,
                                                ir_flash_power_patch_list,
                                                aperture_patch_list,
                                                cdt_menu_patch_list,
                                                pt_patch_list,
                                                rsc_patch_list
                            ],
                            list_of_patch_bases = ["release"], command_file_base = "general",
                            offset2a_source='HAND_PATCH'
                            )


Buidling version 240506P
Extended Filename = /content/drive/MyDrive/local_repositories/unified-btc-reverse/version_table.md
Building .BRN file for Target: BTC-7E
Info::carve: (A) partition offset = 0x96920
Info::carve A size: 0x  160000
Info::carve: (B) partition offset = 0x1f6920
Info::carve B size: 0x   20000
Info::make_burn_file: getting A file system from /content/drive/MyDrive/local_repositories/unified-btc-reverse/targets/btc-7e/A-File-System/offset2.A.wbwl.img
Info::make_burn_file: adding patch_list: dict_keys(['Custom_Ribbon', 'ld_draw_video_scroll_bar_hook', 'ld_draw_video_scroll_bar_hook2', 'ld_clear_video_scroll_bar_hook'])
Info::make_burn_file: adding patch_list: dict_keys(['No_Video_Limit_Store'])
Info::make_burn_file: adding patch_list: dict_keys(['date_local_sprintf', 'time_local_sprintf', 'last_strlen', 'still_logo', 'video_logo', 'load_font_width'])
Info::make_burn_file: adding patch_list: dict_keys(['firmware_ID0', 'firmware_ID1'])
Info::make_burn_file: adding patch_l